### Leitura das métricas logadas

In [66]:
from pathlib import Path
import os
from pprint import pprint

RESULTS_DIR = Path.cwd().parent / 'results'

#### Fase 1

In [67]:
import polars as pl
from tensorboard.backend.event_processing import event_accumulator

In [ ]:
# Carregar os dados dos logs do TensorBoard
def load_events_data(log_dir: str):
    ea = event_accumulator.EventAccumulator(log_dir)
    ea.Reload()

    # Acessar os dados de scalars (loss, accuracy, etc.)
    tags = ea.Tags()['scalars']

    # Criar um dataframe vazio para armazenar os dados
    data_train = {}
    data_test = []

    # Iterar sobre as tags e extrair os valores
    for tag in tags:
        if 'step' not in tag and tag != 'epoch':
            events = ea.Scalars(tag)
            for event in events:
                step = event.step
                value = event.value
                if 'train' in tag:
                    data_train.append([tag, step, value])
                elif 'test' in tag:
                    data_test.append([tag, step, value])

    df_train = pl.DataFrame(data_train, schema=["Tag", "Step", "Value"])
    df_test = pl.DataFrame(data_test, schema=["Tag", "Step", "Value"])

    # Mostrar o DataFrame
    print(df_train)

    # # Salvar o DataFrame como CSV (se necessário)
    # df.write_csv(RESULTS_DIR/f"phase_1/csvs/tensorboard_data_polars_{'_'.join(log_dir.split(os.path.sep)[-3:])}.csv")
    # print(df)

In [ ]:
#          metric x | metric y | ...
#           value1  |  value1  | ...
#           value2  |  value2  | ...
#           value3  |  value3  | ...
#           value4  |  value4  | ...
#           value5  |  value5  | ...

for events_dir in RESULTS_DIR.glob('phase_1/tb_logs/*/repet*/kfolditer*'):
    load_events_data(str(events_dir))  # carrega os datos de uma iteração do kfold

    #TODO: como agregar os dados

/tmp/ipykernel_2013292/2223788853.py:25: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  df_train = pl.DataFrame(data_train, schema=["Tag", "Step", "Value"])
/tmp/ipykernel_2013292/2223788853.py:26: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  df_test = pl.DataFrame(data_test, schema=["Tag", "Step", "Value"])


shape: (46, 3)
┌───────────────────────────┬───────┬────────────┐
│ Tag                       ┆ Step  ┆ Value      │
│ ---                       ┆ ---   ┆ ---        │
│ str                       ┆ i64   ┆ f64        │
╞═══════════════════════════╪═══════╪════════════╡
│ epoch_train_per_image_iou ┆ 2153  ┆ 0.275053   │
│ epoch_train_per_image_iou ┆ 4307  ┆ 0.298951   │
│ epoch_train_per_image_iou ┆ 6461  ┆ 0.301275   │
│ epoch_train_per_image_iou ┆ 8615  ┆ 0.30386    │
│ epoch_train_per_image_iou ┆ 10769 ┆ 0.307091   │
│ …                         ┆ …     ┆ …          │
│ epoch_avg_loss_train      ┆ 4307  ┆ 0.030497   │
│ epoch_avg_loss_train      ┆ 6461  ┆ 0.02482    │
│ epoch_avg_loss_train      ┆ 8615  ┆ 0.023212   │
│ epoch_avg_loss_train      ┆ 10769 ┆ 0.019286   │
│ training_time             ┆ 0     ┆ 595.231873 │
└───────────────────────────┴───────┴────────────┘
shape: (46, 3)
┌───────────────────────────┬───────┬────────────┐
│ Tag                       ┆ Step  ┆ Value      │
│

# Objetivo:

UNet++:
* desempenho médio das iterações do kfold
* média dos desempenhos médios do kfold, considerando as 3 iteraçoes